# Rivals VOD helper
Just insert your URL and then click *Run after* (Ctrl + F10 - Runtime -> Run after) to get your output.

Note: please use either Youtube OR Twitch at once.

### Download from YouTube

In [48]:
#@markdown Insert your Youtube URL to begin. { run: "auto" }
try:
  import pytube
except ModuleNotFoundError:
  subprocess.run(["pip", "-q", "install", "pytube"])
  import pytube

youtube_url = "https://www.youtube.com/watch?v=24YPA0_K8ao" #@param {type:"string"}

if youtube_url:
  yt = pytube.YouTube(youtube_url.replace("watch?v=", ""))
  yt = yt.streams.filter(
      progressive=True, 
      file_extension='mp4'
    ).order_by('resolution').desc().first()

  if not os.path.exists("."):
      os.makedirs(".")

  yt.download(".")

### Download from Twitch

In [3]:
#@markdown Insert your Twitch URL to begin. { run: "auto" }
twitch_url = "" #@param {type:"string"}

if twitch_url:
  !pip -q install twitch-dl
  !twitch-dl download -q source -f mp4 $twitch_url

## Directories handling
Creates snapshots folder and video-specific subfolders.

In [49]:
#@title
# orders files by last added
files = list(filter(os.path.isfile, glob.glob("*")))
files.sort(key=lambda x: ".mp4" in x and os.path.getmtime(x))

# handles situation in which no file is present
if not files:
  raise Exception("An error has occurred: no video files in directory.")

# gets latest file added and sets directory name
input_file = files[-1]
dir_name = input_file.replace(".mp4", "")

# deletes folder if it already exists
try:
  shutil.rmtree(f"snapshots/{dir_name}")
except FileNotFoundError:
  pass

# creates snapshots folder if it doesn't exist
for dir in ["snapshots", f"snapshots/{dir_name}"]:
  try:
    os.mkdir(dir)
  except FileExistsError:
    pass

## Dependencies
Imports dependencies and downloads model from repo.

In [1]:
#@title
import numpy as np
import pandas as pd
import tensorflow as tf 
import os.path
import shutil
import glob
import ipywidgets as widgets
from pathlib import Path
from tensorflow.keras.models import load_model
from IPython.display import display
import subprocess

try:
  import google.colab
  COLAB = True
except ModuleNotFoundError:
  COLAB = False

try:
  import ffmpeg
except ModuleNotFoundError:
  subprocess.run(["pip", "-q", "install", "ffmpeg-python"])
  import ffmpeg

<module 'pytube' from '/usr/local/lib/python3.7/dist-packages/pytube/__init__.py'>


In [2]:
# clones model from repo
!rm -r model
!git clone https://github.com/Impasse52/rivals_vods_helper
!mv ./rivals_vods_helper/model ./model
!rm -r rivals_vods_helper

# loads model from directory
model = load_model("./model")

rm: cannot remove 'model': No such file or directory
Cloning into 'rivals_vods_helper'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 43 (delta 16), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (43/43), done.


## Video to frames conversion
Uses `ffmpeg-python` to convert a video file to a succession of frames from the same video, grabbing one frame each second by default.

In [50]:
#@title
print(f"Processing: {input_file}")
try:
    (ffmpeg.input(input_file)
          .filter('fps', fps=1)
          .output(
              f'snapshots/{dir_name}/%d.jpg', 
              s='426x240',
              start_number=0
          ).run(capture_stdout=True, capture_stderr=True))
except ffmpeg.Error as e:
    print('stdout:', e.stdout.decode('utf8'))
    print('stderr:', e.stderr.decode('utf8'))

Processing: Aether Rising 11  Losers Eights - Kii (Sylvanos) vs Memento (Kragg).mp4


## Frames prediction
Defines `predict_screen` which handles each frame (resizing, normalization, etc.) and then predicts its label.




In [51]:
#@title
def predict_screen(img_dir, labels):
  img = tf.keras.preprocessing.image.load_img(
    img_dir, # inputs directory
    target_size=(224, 224) # resizes images
  )

  # (height, width, channels)
  img_tensor = tf.keras.preprocessing.image.img_to_array(img) 

  # (1, height, width, channels), add a dimension because the model 
  # expects this shape: (batch_size, height, width, channels)
  img_tensor = np.expand_dims(img_tensor, axis=0) 

  # normalization
  img_tensor /= 255. 

  # predicts the label of the test_images
  predictions = model.predict(img_tensor)

  # gets label with highest confidence
  prediction_index = np.argmax(predictions, axis=1)[0]
  label = labels[prediction_index]

  # uses original file's name as output name
  output_name = img_dir.split("/")[-1]

  return {
      output_name: label,
      "confidence": predictions[0][0]
  }

# TODO: expand return dictionary like this:
# return {
#     "label": label,
#     "filename": output_name,
#     "confidence": pred[0][0]
# }

In [52]:
#@title
# labels definition
labels = {
  0: "bracket_view",
  1: "character_selection_screen",
  2: "gameplay",
  3: "stage_selection_screen",
  4: "versus_screen",
  5: "victory_screen"
}

dir = fr"/content/snapshots/{dir_name}" if COLAB else fr"snapshots/{dir_name}"

## Output handling
Creates a subfolder for each label and moves the frame to the appropriate subfolder. Everything is then saved to a .zip file.

In [ ]:
#@title
# creates directory for each value in dict
for l in labels.values():
  try:
    os.mkdir(fr"{dir}/{l}")
  except FileExistsError:
    pass

for f in os.listdir(dir):
  if f not in labels.values():
    # TODO: decide on a threshold which excludes "not good enough" screenshots
    prediction = predict_screen(fr"{dir}/{f}", labels)
    print(prediction)

    pred_dir = prediction[f]
    shutil.move(fr"{dir}/{f}", fr"{dir}/{pred_dir}/{f}")
    # print(fr"{dir}/{f}", fr"{dir}/{pred_dir}/{f}")

In [55]:
#@title
!zip -r "./rivals_predict_output.zip" "./snapshots" 
# !rm -r snapshots

  adding: snapshots/ (stored 0%)
  adding: snapshots/Aether Rising 11  Winners Semis - Kai (Ori) vs Orticoltore (Etalus)/ (stored 0%)
  adding: snapshots/Aether Rising 11  Winners Semis - Kai (Ori) vs Orticoltore (Etalus)/gameplay/ (stored 0%)
  adding: snapshots/Aether Rising 11  Winners Semis - Kai (Ori) vs Orticoltore (Etalus)/gameplay/191.jpg (deflated 0%)
  adding: snapshots/Aether Rising 11  Winners Semis - Kai (Ori) vs Orticoltore (Etalus)/gameplay/697.jpg (deflated 0%)
  adding: snapshots/Aether Rising 11  Winners Semis - Kai (Ori) vs Orticoltore (Etalus)/gameplay/367.jpg (deflated 0%)
  adding: snapshots/Aether Rising 11  Winners Semis - Kai (Ori) vs Orticoltore (Etalus)/gameplay/25.jpg (deflated 0%)
  adding: snapshots/Aether Rising 11  Winners Semis - Kai (Ori) vs Orticoltore (Etalus)/gameplay/133.jpg (deflated 0%)
  adding: snapshots/Aether Rising 11  Winners Semis - Kai (Ori) vs Orticoltore (Etalus)/gameplay/481.jpg (deflated 0%)
  adding: snapshots/Aether Rising 11  Winne